## Prozorro API

- https://github.com/openprocurement/openprocurement.api/blob/master/docs/source/tenders.rst
- https://prozorro-api-docs.readthedocs.io/en/master/authentication.html#api-keys
- [basic rules of using Prozorro API](http://api-docs.openprocurement.org/en/latest/tutorial.html)
- [Python client](https://github.com/openprocurement/openprocurement.client.python/tree/master/openprocurement_client)

In [0]:
from IPython.display import JSON 
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.parse as url
#import subprocess #для Ipython notebook

In [0]:
from pathlib import Path
GDRIVE_PATH='/content/gdrive/My Drive/code/Prozorro/'
if not Path(GDRIVE_PATH).exists():
    from google.colab import drive
    drive.mount('/content/gdrive/')

In [0]:
#https://www.reddit.com/r/IPython/comments/34t4m7/lpt_print_json_in_collapsible_format_in_ipython/
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

## Робота з API Prozorro.

In [0]:
#endpoint = 'http://lb-api-sandbox.prozorro.gov.ua/api/2.5' Prozorro sandbox API

In [0]:
endpoint = 'http://public.api.openprocurement.org/api/0'

In [0]:
t = datetime.now()

In [16]:
url.quote(t.isoformat())

'2020-05-20T20%3A17%3A25.064470'

In [0]:
#r = requests.get(endpoint + '/tenders?offset=' + url.quote(t.isoformat()))
r = requests.get(endpoint + '/tenders', params={
    'offset': t.isoformat(),
    'descending': 1,
})
if r.status_code != 200:
    print('Request failed, status_code=', r.status_code)

rd = r.json()

In [18]:
r.url

'https://public.api.openprocurement.org/api/0/tenders?offset=2020-05-20T20%3A17%3A25.064470&descending=1'

In [19]:
RenderJSON(rd)

In [20]:
tender_ids = [tender['id'] for tender in rd['data']]
tender_ids

['e842cd3adb884abcb51beb8bfb93e58d',
 '884677f7e5044552b5074717260f11a0',
 '005f7e7892334675b2a3d76bca1ce1d5',
 'a83c79737c9149219172603444d2c60a',
 '6560e5ca49c841ce9ddae0e3471a5c51',
 'a2a69ba9881547c4ae24c85e300745fb',
 '2c5c67ae38774c1dadf492f39eda61f7',
 '5cfeeaeca7e94d30a50c5038bc854f24',
 '23f9b8419f454569a7526e31000f8b5b',
 '8ec7d23195f24776b6f1a62695ebd90b',
 '2dca316b53ce4c4a878c42283a258ba1',
 '232805421341453b8b0868f199ab2ed3',
 '249ff3f17b834448982e95c96d7b73a8',
 '1f23c65243eb43688127781577063e88',
 'abe80f40abb6405d87fd46776f830dec',
 '72a76acf0abe4bfcacd5a8e8b201ace9',
 '9e12c5a8bdf84d899cc80ff96ee4a446',
 'd6294585d07c423f89c55d3ed1461adf',
 'b05891827d94471d999165012572dd0c',
 '66532500f5aa4d5db5eb35f830993d3d',
 'db645c8eea0447d599e5ce5311fa7f30',
 '9f06d6a1718d465dabbd6142e9bb719d',
 'c828506a97254db49d05bdb63f853a35',
 'fd3c6a0803154f3f9511af9c75925708',
 'f0926f7e162743afb1a6cd21ca2c3afe',
 'fd713f4ec4944710a4a680790cec158e',
 '2669a0c681c74c10a963f71c30fc0316',
 

## Приклад витягування даних про конкретний тендер

In [0]:
tender_id = 'c20bcc41e1c548489989e711575a5c5c' 
#tender_id = rd['data'][8]['id']

In [12]:
r1 = requests.get(f'{endpoint}/tenders/{tender_id}')
print('status', r1.status_code)

status 200


In [13]:
RenderJSON(r1.json()) # для Ipython notebook

In [21]:
rd1 = r1.json()['data']
tender_title = rd1['title']
for item in rd1['items']:
    clss = item['classification']
    print(f"tender_title={tender_title}, item_desc={item['description']}")
    print(f"classification={clss['scheme']} {clss['id']}")
    print()

tender_title=бензин А-92, паливо дизельне, item_desc=бензин А-92
classification=ДК021 09130000-9

tender_title=бензин А-92, паливо дизельне, item_desc=паливо дизельне
classification=ДК021 09130000-9



## Витягування всіх тендерів

In [0]:
tenders = []

for tender_id in tender_ids:
    r = requests.get(f'{endpoint}/tenders/{tender_id}')
    if r.status_code != 200:
        print('status', r.status_code)
        continue
    data = r.json()['data']
    tenders.append(data)
    #tender_title = data['title']
    #tender_desc = data.get('description')

In [23]:
tenders

Output hidden; open in https://colab.research.google.com to view.

In [24]:
tenders[0].keys()

dict_keys(['procurementMethod', 'numberOfBids', 'mainProcurementCategory', 'complaintPeriod', 'enquiryPeriod', 'submissionMethod', 'complaints', 'awardCriteria', 'questions', 'owner', 'qualifications', 'id', 'plans', 'documents', 'title', 'qualificationPeriod', 'tenderID', 'auctionPeriod', 'guarantee', 'dateModified', 'status', 'tenderPeriod', 'procurementMethodType', 'title_en', 'date', 'milestones', 'minimalStep', 'items', 'bids', 'value', 'procuringEntity'])

In [25]:
tenders[0]['items']

[{'additionalClassifications': [{'description': 'Київ - Чернігів - Нові Яриловичі (на м. Гомель)',
    'id': 'M-01',
    'scheme': 'UA-ROAD'}],
  'classification': {'description': 'Ремонт доріг',
   'id': '45233142-6',
   'scheme': 'ДК021'},
  'deliveryAddress': {'countryName': 'Україна',
   'locality': 'Чернігів',
   'postalCode': '14005',
   'region': 'Чернігівська область',
   'streetAddress': 'вул.Київська,17'},
  'deliveryDate': {'endDate': '2020-12-31T00:00:00+02:00'},
  'description': 'Поточний середній ремонт автомобільної дороги загального користування державного значення М-01 Київ-Чернігів-Нові Яриловичі (на м. Гомель) на ділянці км 192+000 — км 203+560 (окремими ділянками), (45233142-6 Ремонт доріг)',
  'description_en': 'Current average repair of the public highway M-01 Kyiv-Chernihiv-Novi Yarilovichi (in Gomel) in km 192 + 000 - km 203 + 560 (in separate sections), (45233142-6 Road repair)',
  'id': 'a324e2af1c8f4242a8ff40bfcbaa9ab1',
  'quantity': 7000.0,
  'unit': {'code

In [26]:
tenders[0]['tenderID']

'UA-2020-02-17-002924-a'

In [27]:
titles = []
descs = []
idescs = []
iunits = []
iclasses = []
ids = []
tenderIDs = []
 
for data in tenders:
    for item in data['items']:
        idescs.append(item['description'])
        iunits.append(item['unit']['name'])
        iclasses.append(item['classification']['description']+' '+item['classification']['id'])
        titles.append(data['title'])
        descs.append(data.get('description'))
        ids.append(data['id'])
        tenderIDs.append(data['tenderID'])

df = pd.DataFrame({
    'id': ids,
    'tenderID': tenderIDs,
    'title': titles,
    'description': descs, 
    'item_desc': idescs, 
    'item_unit': iunits,
    'item_class': iclasses,
})

df.head(20)

,id,tenderID,title,description,item_desc,item_unit,item_class
0,e842cd3adb884abcb51beb8bfb93e58d,UA-2020-02-17-002924-a,Поточний середній ремонт автомобільної дороги ...,None,Поточний середній ремонт автомобільної дороги ...,метри,Ремонт доріг 45233142-6
1,884677f7e5044552b5074717260f11a0,UA-2020-05-20-006770-c,Дюбель з шурупом,дюбель з шурупом 6*40мм,Дюбель з шурупом,штуки,Кріпильні деталі 44530000-4
2,005f7e7892334675b2a3d76bca1ce1d5,UA-2020-05-20-006767-c,Мастильні засоби,якісні та дешеві,Мастильні засоби,штуки,Мастильні засоби 09210000-4
3,a83c79737c9149219172603444d2c60a,UA-2020-05-20-003285-a,Індивідуальне обмундирування,None,засіб індивідуального захисту (тип Маска),шт,Індивідуальне обмундирування 35810000-5
4,6560e5ca49c841ce9ddae0e3471a5c51,UA-2020-03-19-004177-b,Апаратура для підтримування фізіологічних функ...,,Системи кохлеарної імплантації призначаються д...,штуки,Апаратура для підтримування фізіологічних функ...
5,a2a69ba9881547c4ae24c85e300745fb,UA-2020-04-13-001059-b,Кейтерингові послуги,None,Кейтерингові послуги,од.,Кейтерингові послуги 55520000-1
6,2c5c67ae38774c1dadf492f39eda61f7,UA-2020-05-20-000231-b,Паливна деревина м'яких та твердих порід,None,Паливна деревина м'яких порід,м.куб,Деревина 03410000-7
7,2c5c67ae38774c1dadf492f39eda61f7,UA-2020-05-20-000231-b,Паливна деревина м'яких та твердих порід,None,Паливна деревина твердих порід,м.куб,Деревина 03410000-7
8,5cfeeaeca7e94d30a50c5038bc854f24,UA-2020-05-20-003863-a,Мікрофони та гучномовці,None,Мікрофони та гучномовці,шт,Мікрофони та гучномовці 32340000-8
9,23f9b8419f454569a7526e31000f8b5b,UA-2020-05-20-003843-a,Будівельні товари,Відповідно до змісту договору та нормативно-пр...,Будівельні товари,штуки,Будівельні товари 44420000-0


In [0]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html

df1 = pd.json_normalize(tenders, 'items', meta=['title', 'id', 'tenderID'], record_prefix='items.')

df1 = df1[['title', 'id', 'tenderID',
           'items.description', 'items.unit.name', 'items.unit.code', 
           'items.classification.description', 'items.classification.id',
          ]]

In [29]:
df1.sort_values(by=['id'])

,title,id,tenderID,items.description,items.unit.name,items.unit.code,items.classification.description,items.classification.id
2,Мастильні засоби,005f7e7892334675b2a3d76bca1ce1d5,UA-2020-05-20-006767-c,Мастильні засоби,штуки,H87,Мастильні засоби,09210000-4
206,Засоби для догляду за руками та нігтями,047ac25a0e5d4cb4ae3121dc58aa9775,UA-2020-05-07-000661-b,Засоби для догляду за руками та нігтями (Антис...,штуки,H87,Засоби для догляду за руками та нігтями,33740000-9
185,Виконання комплексу спеціалізованих робіт з пі...,062888fe1dc94a3b90a2eaae7ce92a34,UA-2020-04-17-006969-a,Виконання комплексу спеціалізованих робіт з пі...,послуга,E48,Послуги з обслуговування наземних видів трансп...,63710000-9
77,"Послуги з ремонту, технічного обслуговування п...",0654068a9ebf4fdba6e775be921e78e9,UA-2020-05-15-002447-b,Середній ремонт авіаційних двигунів Д-30КП-2,одиниця,E50,"Послуги з ремонту, технічного обслуговування п...",50210000-0
178,Дрова паливні твердої породи,08c674c2b9ec49618402823bc7a4ee93,UA-2020-04-13-000705-c,Дрова паливні твердої породи,м.куб,MTQ,Деревина,03410000-7
...,...,...,...,...,...,...,...,...
40,Обладнання для закладів громадського харчуванн...,fd3c6a0803154f3f9511af9c75925708,UA-2020-05-08-003735-b,"Котел (казан) електричний стравоварний, об'єм ...",одиниця,E50,Обладнання для їдалень,39312200-4
41,Обладнання для закладів громадського харчуванн...,fd3c6a0803154f3f9511af9c75925708,UA-2020-05-08-003735-b,"Котел (казан) електричний стравоварний, об'єм ...",одиниця,E50,Обладнання для їдалень,39312200-4
34,Обладнання для закладів громадського харчуванн...,fd3c6a0803154f3f9511af9c75925708,UA-2020-05-08-003735-b,"Котел (казан) електричний стравоварний, об'єм ...",одиниця,E50,Обладнання для їдалень,39312200-4
39,Обладнання для закладів громадського харчуванн...,fd3c6a0803154f3f9511af9c75925708,UA-2020-05-08-003735-b,"Котел (казан) електричний стравоварний, об'єм ...",одиниця,E50,Обладнання для їдалень,39312200-4


## Робота з датафремом майбутніх закупок

In [0]:
Plans = pd.read_excel(GDRIVE_PATH+'/Plans-vs-OCDS.xlsx')

In [147]:
Plans.head()

,Ідентифікатор пункт плану,Пункт плану (розширений),Примітки,Сума пунктів плану,Процедура закупівлі,...,Назва організатора,Рік-Місяць планованого оголошення,Рік-Місяць доставки,К-сть місяців з публікації до оголошення,Дата публікації пункту плану,Класифікація CPV
0,UA-P-2020-04-28-000015-b,інженерно-геологічних вишукувань по об’єкту: «...,-,4900.0,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-28-0...,"Меморіальний музей тоталітарних режимів ""Терит...",2020-кві,-,0,2020-04-28,71510000-6 Послуги у сфері інженерно-геологічн...
1,UA-P-2020-04-27-004544-b,Послуги: Поточний ремонт ливневої каналізаційн...,Послуги: Поточний ремонт ливневої каналізаційн...,49046.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Личаківська районна адміністрація Львівської м...,2020-кві,-,0,2020-04-27,"45230000-8 Будівництво трубопроводів, ліній зв..."
2,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",-,9947.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Ліцей № 51 ім. Івана Франка Львівської міської...,2020-кві,-,0,2020-04-27,71520000-9 Послуги з нагляду за виконанням буд...
3,UA-P-2020-04-27-004503-b,ДК 021:2015 — 50112000-3 - Послуги з ремонту і...,-,6220.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,50112000-3 Послуги з ремонту і технічного обсл...
4,UA-P-2020-04-27-004470-b,"""Ремонтно-реставраційні роботи будинку на вул....","Категорія замовника. підприємства, установи, о...",9970759.2,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,"Львівське комунальне підприємство ""Агенція рес...",2020-кві,-,0,2020-04-27,45453000-7 Капітальний ремонт і реставрація


## Відфільтровування описів тендерів, які містять цільове значення 

In [32]:
# Examples
s1 = pd.Series(['Mouse', 'dog', 'house and parrot', 23, np.NaN])
s1.str.contains('og', regex=False)

pd.Series(['foo', 'fuz', np.nan]).str.replace('f.', 'ba', regex=True)

0    bao
1    baz
2    NaN
dtype: object

In [148]:
Plans.columns

Index(['Ідентифікатор пункт плану', 'Пункт плану (розширений)', 'Примітки',
       'Сума пунктів плану', 'Процедура закупівлі', '...',
       'Назва організатора', 'Рік-Місяць планованого оголошення',
       'Рік-Місяць доставки', 'К-сть місяців з публікації до оголошення',
       'Дата публікації пункту плану', 'Класифікація CPV'],
      dtype='object')

In [149]:
Plans.shape

(15384, 12)

In [150]:
len(Plans['Пункт плану (розширений)'].unique())

11861

In [0]:
Plans['Пункт плану (розширений)'] = Plans['Пункт плану (розширений)'].str.replace('\d{8}[-]\d', '', regex=True)

In [0]:
Plans['Пункт плану (розширений)'] = Plans['Пункт плану (розширений)'].str.replace('\w{2} \d{3}:\d{4}', '', regex=True)

In [0]:
Plans['Пункт плану (розширений)'] = Plans['Пункт плану (розширений)'].str.replace('[|]', '', regex=True)

In [0]:
Plans['Пункт плану (розширений)'] = Plans['Пункт плану (розширений)'].str.replace('[-—:]', '', regex=True)

In [155]:
Plans.head(50)

,Ідентифікатор пункт плану,Пункт плану (розширений),Примітки,Сума пунктів плану,Процедура закупівлі,...,Назва організатора,Рік-Місяць планованого оголошення,Рік-Місяць доставки,К-сть місяців з публікації до оголошення,Дата публікації пункту плану,Класифікація CPV
0,UA-P-2020-04-28-000015-b,інженерногеологічних вишукувань по об’єкту «Ре...,-,4900.00,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-28-0...,"Меморіальний музей тоталітарних режимів ""Терит...",2020-кві,-,0,2020-04-28,71510000-6 Послуги у сфері інженерно-геологічн...
1,UA-P-2020-04-27-004544-b,Послуги Поточний ремонт ливневої каналізаційно...,Послуги: Поточний ремонт ливневої каналізаційн...,49046.00,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Личаківська районна адміністрація Львівської м...,2020-кві,-,0,2020-04-27,"45230000-8 Будівництво трубопроводів, ліній зв..."
2,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",-,9947.00,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Ліцей № 51 ім. Івана Франка Львівської міської...,2020-кві,-,0,2020-04-27,71520000-9 Послуги з нагляду за виконанням буд...
3,UA-P-2020-04-27-004503-b,Послуги з ремонту і технічного обслуговува...,-,6220.00,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,50112000-3 Послуги з ремонту і технічного обсл...
4,UA-P-2020-04-27-004470-b,"""Ремонтнореставраційні роботи будинку на вул. ...","Категорія замовника. підприємства, установи, о...",9970759.20,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,"Львівське комунальне підприємство ""Агенція рес...",2020-кві,-,0,2020-04-27,45453000-7 Капітальний ремонт і реставрація
5,UA-P-2020-04-27-004445-b,Антисептичні та дезінфекційні засоби,ДК 021:2015: 33631600-8 — Антисептичні та дезі...,4351.80,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Личаківська районна адміністрація Львівської м...,2020-кві,-,0,2020-04-27,33631600-8 Антисептичні та дезінфекційні засоби
6,UA-P-2020-04-27-004398-b,Металопластикові вироби,-,48426.94,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Державний професійно-технічний навчальний закл...,2020-кві,-,0,2020-04-27,"44221000-5 Вікна, двері та супутні вироби"
7,UA-P-2020-04-27-004370-b,Нагляд за будівельними роботами ТН на капремо...,-,13117.67,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,71247000-1 Нагляд за будівельними роботами
8,UA-P-2020-04-27-004367-b,Нагляд за будівельними роботами ТН на капремо...,-,17463.77,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,71247000-1 Нагляд за будівельними роботами
9,UA-P-2020-04-27-004362-b,Нагляд за будівельними роботами ТН на капремо...,-,10663.04,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,71247000-1 Нагляд за будівельними роботами


In [0]:
NUM_PRED = 10

In [157]:
template_q = '''
    {
      Predictions {
        Classification(
          page: {limit: %d},
          filters:[
          %%s
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }
''' % NUM_PRED

filters = '''
{eq: {field: "tender.title", value: "Плитка керамічна"}},
{eq: {field: "tender.description", value: "плитка керамічна"}},
{eq: {field: "item.description", value: "Плитка керамічна Gress Атем Veneto, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
{eq: {field: "item.unit.id", value: "MTK"}},
{gte: {field: "probability", value: "0.0002"}}'''

print(template_q % filters)


    {
      Predictions {
        Classification(
          page: {limit: 10},
          filters:[
          
{eq: {field: "tender.title", value: "Плитка керамічна"}},
{eq: {field: "tender.description", value: "плитка керамічна"}},
{eq: {field: "item.description", value: "Плитка керамічна Gress Атем Veneto, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
{eq: {field: "item.unit.id", value: "MTK"}},
{gte: {field: "probability", value: "0.0002"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }



In [158]:
def generate_filters(tender_desc, tender_title=None, item_desc=None, item_unit=None, prob=0.0002):
    fs = []
    def sanitize(s):
        return s.replace('"', '«').replace('\n', ' ').replace('\\', r'\\')
    
    if tender_title is not None:
        fs.append('{eq: {field: "tender.title", value: "%s"}}' % sanitize(tender_title))
    
    fs.append('{eq: {field: "tender.description", value: "%s"}}' % sanitize(tender_desc))
    
    if item_desc is not None:
        fs.append('{eq: {field: "item.description", value: "%s"}}' % sanitize(item_desc))
        
    if item_unit is not None:
        fs.append('{eq: {field: "item.unit.id", value: "%s"}}' % sanitize(item_unit))
        
    fs.append('{gte: {field: "probability", value: "%f"}}' % prob)
    return ",\n".join(fs)

print(generate_filters('плитка керамічна', 
                       tender_title='Плитка керамічна',
                       item_desc='Плитка керамічна Gress Атем "Veneto", для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова'))
    
        

{eq: {field: "tender.title", value: "Плитка керамічна"}},
{eq: {field: "tender.description", value: "плитка керамічна"}},
{eq: {field: "item.description", value: "Плитка керамічна Gress Атем «Veneto«, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
{gte: {field: "probability", value: "0.000200"}}


In [159]:
#only for colab
!pip3 install gql

In [0]:
#use only for colab
from getpass import getpass 

In [161]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from json import dumps
base_url = 'https://ocdsanalytics.com/ua/prozorro/graphql'

#for ipython notebook:
#with open('./graphql_token') as f:
#    auth_token = f.read().strip()

#for colab:
auth_token = getpass('Your graphql token, please: ')

transport_ocds = RequestsHTTPTransport(url = base_url, 
                                       headers = {'Authorization':'Bearer '+auth_token}, 
                                       use_json=True)

client = Client(transport=transport_ocds)

Your graphql token, please: ··········


In [0]:
data = []

In [163]:
import time 

print('before')
time.sleep(5)
print('after')

before
after


In [0]:
def flatten_entities(r):
    d = {}
    def flatten(i):
        d['id'+str(i+1)] = r[i]['entity']['id']
        #d['desc'+str(i+1)] = r[i]['entity']['description']
        d['prob'+str(i+1)] = r[i]['probability']

    for i in range(0, min(NUM_PRED, len(r))):
        flatten(i)

    return d

In [165]:
flatten_entities([
        {'entity': {
                'id': '50112000-3', 
                'description': 'Послуги з ремонту і технічного обслуговування автомобілів', 
                'scheme': 'CPV'},
         'probability': 0.9890542}, 
        {'entity': {
                'id': '50110000-9', 
                'description': 'Послуги з ремонту і технічного обслуговування мототранспортних засобів і супутнього обладнання',
                'scheme': 'CPV'},
         'probability': 0.010862832}
    ])

{'id1': '50112000-3',
 'id2': '50110000-9',
 'prob1': 0.9890542,
 'prob2': 0.010862832}

In [166]:
%%time
params = {'limit':1} 

#try:
for i in range(100): #передавання лише перших n-рядків з Plans для тестування;
#for i in range(len(data), len(Plans)): #передавання всіх рядків в Plans
    tender_desc = Plans['Пункт плану (розширений)'][i]
    CPV_Plans = Plans['Класифікація CPV'][i]
    Plan_identificator = Plans['Ідентифікатор пункт плану'][i]
    
    q = template_q % generate_filters(tender_desc)
    print(q)
    q = gql(q)

    r = client.execute(q, variable_values=params)
    r = r['Predictions']['Classification']['values']
    row = flatten_entities(r)
    
    row['CPV з плану'] = CPV_Plans
    row['Пункт плану (розширений)'] = tender_desc
    row['Ідентифікатор пункту плану'] = Plan_identificator
    #print(row)

    #print(f"prob={p:0.2f}, id={r0['id']}, scheme={r0['scheme']}, desc={r0['description']}")
    data.append(row)
    time.sleep(0.2)
#except:
#    subprocess.call(['mplayer', '/home/qgelena/Music/Scary-stinger-dun-dun-dun.mp3'])
#    raise
    
#subprocess.call(['mplayer', '/home/qgelena/Music/Cat-meows-sounds.wav'])


    {
      Predictions {
        Classification(
          page: {limit: 10},
          filters:[
          {eq: {field: "tender.description", value: "інженерногеологічних вишукувань по об’єкту «Реконструкція Меморіальним музеєм тоталітарних режимів «Територія Терору» нежитлових приміщень на вул. Старій, 13 під відділ «Музей малолітніх політичних в'язнів і репресованих»  "}},
{gte: {field: "probability", value: "0.000200"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }


    {
      Predictions {
        Classification(
          page: {limit: 10},
          filters:[
          {eq: {field: "tender.description", value: "Послуги Поточний ремонт ливневої каналізаційної мережі на пл.Петрушевича 3 у м.Львові [за кодом CPV за  (CPV)    (Будівництво трубопроводіва, ліній звязку та електропередач, шосе, доріг, аеродромів і залізничних доріг; вирівню

In [0]:
# Для ручного додавання колонки, це вже робиться в циклі;
#df1['Plan_identificator'] = Plans['Ідентифікатор пункт плану'] 
#df1['description_plan_item'] = Plans['Пункт плану (розширений)']

In [0]:
df1 = pd.DataFrame(data)

In [168]:
df1.columns

Index(['id1', 'prob1', 'id2', 'prob2', 'id3', 'prob3', 'id4', 'prob4', 'id5',
       'prob5', 'id6', 'prob6', 'id7', 'prob7', 'id8', 'prob8', 'id9', 'prob9',
       'id10', 'prob10', 'CPV з плану', 'Пункт плану (розширений)',
       'Ідентифікатор пункту плану'],
      dtype='object')

In [169]:
df1.head()

,id1,prob1,id2,prob2,id3,prob3,id4,prob4,id5,prob5,id6,prob6,id7,prob7,id8,prob8,id9,prob9,id10,prob10,CPV з плану,Пункт плану (розширений),Ідентифікатор пункту плану
0,79713000-5,0.244532,79710000-4,0.065498,71240000-2,0.045981,45454000-4,0.044919,71355000-1,0.044411,45000000-7,0.043375,71510000-6,0.040920,71250000-5,0.037525,79711000-1,0.034360,71320000-7,0.033608,71510000-6 Послуги у сфері інженерно-геологічн...,інженерногеологічних вишукувань по об’єкту «Ре...,UA-P-2020-04-28-000015-b
1,45230000-8,0.999917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45230000-8 Будівництво трубопроводів, ліній зв...",Послуги Поточний ремонт ливневої каналізаційно...,UA-P-2020-04-27-004544-b
2,45453000-7,0.412781,45450000-6,0.153498,45261000-4,0.152376,45260000-7,0.059486,45200000-9,0.047588,45400000-1,0.034377,45210000-2,0.025401,45230000-8,0.012260,45300000-0,0.012021,45000000-7,0.009533,71520000-9 Послуги з нагляду за виконанням буд...,"""Капітальний ремонт даху з водовідведенням у ...",UA-P-2020-04-27-004528-b
3,50112000-3,0.517599,50110000-9,0.414187,50530000-9,0.018717,50112200-5,0.018097,50112100-4,0.009310,50114000-7,0.005127,50116000-1,0.004195,50100000-6,0.004024,50000000-5,0.003200,63710000-9,0.000843,50112000-3 Послуги з ремонту і технічного обсл...,Послуги з ремонту і технічного обслуговува...,UA-P-2020-04-27-004503-b
4,45260000-7,0.197195,45230000-8,0.151064,45310000-3,0.091480,45330000-9,0.088683,45453000-7,0.085086,45450000-6,0.047697,45420000-7,0.029505,45261000-4,0.023586,45343100-4,0.021437,45000000-7,0.016413,45453000-7 Капітальний ремонт і реставрація,"""Ремонтнореставраційні роботи будинку на вул. ...",UA-P-2020-04-27-004470-b


In [170]:
cols = ['Ідентифікатор пункту плану', 'Пункт плану (розширений)', 'CPV з плану',]

for i in range(1, NUM_PRED+1):
    cols.extend(['id'+str(i), 'prob'+str(i)]) #'desc'+str(i), 

print(cols)

['Ідентифікатор пункту плану', 'Пункт плану (розширений)', 'CPV з плану', 'id1', 'prob1', 'id2', 'prob2', 'id3', 'prob3', 'id4', 'prob4', 'id5', 'prob5', 'id6', 'prob6', 'id7', 'prob7', 'id8', 'prob8', 'id9', 'prob9', 'id10', 'prob10']


In [0]:
df1 = df1[cols]

In [174]:
df1.head(10)

,Ідентифікатор пункту плану,Пункт плану (розширений),CPV з плану,id1,prob1,id2,prob2,id3,prob3,id4,prob4,id5,prob5,id6,prob6,id7,prob7,id8,prob8,id9,prob9,id10,prob10
0,UA-P-2020-04-28-000015-b,інженерногеологічних вишукувань по об’єкту «Ре...,71510000-6 Послуги у сфері інженерно-геологічн...,79713000-5,0.244532,79710000-4,0.065498,71240000-2,0.045981,45454000-4,0.044919,71355000-1,0.044411,45000000-7,0.043375,71510000-6,0.040920,71250000-5,0.037525,79711000-1,0.034360,71320000-7,0.033608
1,UA-P-2020-04-27-004544-b,Послуги Поточний ремонт ливневої каналізаційно...,"45230000-8 Будівництво трубопроводів, ліній зв...",45230000-8,0.999917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",71520000-9 Послуги з нагляду за виконанням буд...,45453000-7,0.412781,45450000-6,0.153498,45261000-4,0.152376,45260000-7,0.059486,45200000-9,0.047588,45400000-1,0.034377,45210000-2,0.025401,45230000-8,0.012260,45300000-0,0.012021,45000000-7,0.009533
3,UA-P-2020-04-27-004503-b,Послуги з ремонту і технічного обслуговува...,50112000-3 Послуги з ремонту і технічного обсл...,50112000-3,0.517599,50110000-9,0.414187,50530000-9,0.018717,50112200-5,0.018097,50112100-4,0.009310,50114000-7,0.005127,50116000-1,0.004195,50100000-6,0.004024,50000000-5,0.003200,63710000-9,0.000843
4,UA-P-2020-04-27-004470-b,"""Ремонтнореставраційні роботи будинку на вул. ...",45453000-7 Капітальний ремонт і реставрація,45260000-7,0.197195,45230000-8,0.151064,45310000-3,0.091480,45330000-9,0.088683,45453000-7,0.085086,45450000-6,0.047697,45420000-7,0.029505,45261000-4,0.023586,45343100-4,0.021437,45000000-7,0.016413
5,UA-P-2020-04-27-004445-b,Антисептичні та дезінфекційні засоби,33631600-8 Антисептичні та дезінфекційні засоби,33631600-8,0.420096,33630000-5,0.190730,33600000-6,0.109936,24455000-8,0.107754,24450000-3,0.089788,33631000-2,0.033706,24400000-8,0.008875,33741300-9,0.007319,33740000-9,0.005148,33000000-0,0.003557
6,UA-P-2020-04-27-004398-b,Металопластикові вироби,"44221000-5 Вікна, двері та супутні вироби",44221000-5,0.380415,44220000-8,0.244600,44210000-5,0.088449,19520000-7,0.063580,44221100-6,0.054114,44200000-2,0.023929,44160000-9,0.023448,44310000-6,0.020361,44110000-4,0.011547,19500000-1,0.006274
7,UA-P-2020-04-27-004370-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.326271,45453000-7,0.187396,45000000-7,0.143045,71520000-9,0.130678,71248000-8,0.031905,71240000-2,0.024127,99999999-9,0.021558,45220000-5,0.021279,45450000-6,0.020546,45230000-8,0.016233
8,UA-P-2020-04-27-004367-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.390795,45453000-7,0.174483,45000000-7,0.152054,71520000-9,0.077016,45220000-5,0.039711,99999999-9,0.024870,45450000-6,0.022052,71000000-8,0.019270,45233000-9,0.014037,45230000-8,0.012545
9,UA-P-2020-04-27-004362-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.326271,45453000-7,0.187396,45000000-7,0.143045,71520000-9,0.130678,71248000-8,0.031905,71240000-2,0.024127,99999999-9,0.021558,45220000-5,0.021279,45450000-6,0.020546,45230000-8,0.016233


In [132]:
df1[['prob1', 'prob2','prob3', 'prob4', 'prob5', 'prob6', 'prob7', 'prob8', 'prob9', 'prob10']].sum()

prob1     54.371751
prob2     12.996697
prob3      7.369797
prob4      4.708013
prob5      3.083079
prob6      2.246177
prob7      1.704346
prob8      1.409280
prob9      1.119944
prob10     0.932227
dtype: float64

Pandas: sum DataFrame rows for given columns

[link text](https://stackoverflow.com/questions/25748683/pandas-sum-dataframe-rows-for-given-columns)

df['e'] = df.sum(axis=1)

df

```
# This is formatted as code
```



In [145]:
df1['sum_prob'] = df1[['prob1', 'prob2','prob3', 'prob4', 'prob5', 'prob6', 'prob7', 'prob8', 'prob9', 'prob10']].sum(axis=1)
df1.head(5)

,description_plan_item,Plan_identificator,CPV_Plans,id1,prob1,id2,prob2,id3,prob3,id4,prob4,id5,prob5,id6,prob6,id7,prob7,id8,prob8,id9,prob9,id10,prob10,sum_prob
0,інженерно-геологічних вишукувань по об’єкту: «...,UA-P-2020-04-28-000015-b,71510000-6 Послуги у сфері інженерно-геологічн...,71510000-6,0.260197,71000000-8,0.103378,45000000-7,0.082351,71240000-2,0.058228,71350000-6,0.057840,45454000-4,0.053863,71250000-5,0.046817,71320000-7,0.042009,45200000-9,0.028162,71355000-1,0.024628,0.757472
1,Послуги: Поточний ремонт ливневої каналізаційн...,UA-P-2020-04-27-004544-b,"45230000-8 Будівництво трубопроводів, ліній зв...",45230000-8,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2,"""Капітальний ремонт даху з водовідведенням у ...",UA-P-2020-04-27-004528-b,71520000-9 Послуги з нагляду за виконанням буд...,45453000-7,0.412781,45450000-6,0.153498,45261000-4,0.152376,45260000-7,0.059486,45200000-9,0.047588,45400000-1,0.034377,45210000-2,0.025401,45230000-8,0.012260,45300000-0,0.012021,45000000-7,0.009533,0.919322
3,ДК 021:2015 — 50112000-3 - Послуги з ремонту і...,UA-P-2020-04-27-004503-b,50112000-3 Послуги з ремонту і технічного обсл...,50112000-3,0.989054,50110000-9,0.010863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999917
4,"""Ремонтно-реставраційні роботи будинку на вул....",UA-P-2020-04-27-004470-b,45453000-7 Капітальний ремонт і реставрація,45453000-7,0.420955,45450000-6,0.164542,45260000-7,0.115985,45453100-8,0.099865,45000000-7,0.072386,99999999-9,0.013833,45262000-1,0.011703,45210000-2,0.010682,45261910-6,0.010564,50720000-8,0.009695,0.930210


In [175]:
len(df1['Ідентифікатор пункту плану'].unique())

100

In [0]:
from pandas import ExcelWriter

writer = ExcelWriter(GDRIVE_PATH+'ocds_analytics_lviv_data_raw.xlsx')
df1.to_excel(writer,'Sheet1', index=False)
writer.save()

In [0]:
df1.to_excel?

In [0]:
df1.to_pickle("./out_data.pkl")

## OCDS Analytics GraphQL API

In [142]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from json import dumps
base_url = 'https://ocdsanalytics.com/ua/prozorro/graphql'

with open('./graphql_token') as f:
    auth_token = f.read().strip()

transport_ocds = RequestsHTTPTransport(url = base_url, 
                                       headers = {'Authorization':'Bearer '+auth_token}, 
                                       use_json=True)

client = Client(transport=transport_ocds)

FileNotFoundError: ignored

In [0]:
params = {'limit':1} 
query = gql('''
    query someVariables($limit:Int!) {
      Predictions {
        Unit(
          page: {limit: $limit},
          filters:[
            {eq: {field: "tender.title", value: "Системи та пристрої нагляду та охорони (Двоканальний автомобільний відеореєстратор)"}},
            {eq: {field: "tender.description", value: "Системи та пристрої нагляду та охорони (Двоканальний автомобільний відеореєстратор)"}},
            {eq: {field: "item.description", value: "Двоканальний автомобільний відеореєстратор"}},
            {eq: {field: "item.classification.id", value: "35120000-1"}},
            {gte: {field: "probability", value: "0.02"}}
          ]){
          values{
            entity{
              id
              name
              symbol
            }
            probability
          }
        }
      }
    }

''')
print(dumps(client.execute(query, variable_values=params), indent= 4))


{
    "Predictions": {
        "Unit": {
            "values": [
                {
                    "entity": {
                        "id": "H87",
                        "name": "\u0448\u0442\u0443\u043a\u0438",
                        "symbol": "\u0448\u0442."
                    },
                    "probability": 0.57209986
                }
            ]
        }
    }
}


In [0]:
import gql.dsl as gqld

In [0]:
q = gqld.DSLField?

In [0]:
q = gqld.DSLField

In [0]:
q.variable_definitions

[VariableDefinition(variable=Variable(name=Name(value='limit')), type=NonNullType(type=NamedType(name=Name(value='Int'))), default_value=None)]

In [0]:
query = gql('''
    {
      Predictions {
        Classification(
          page: {limit: 3},
          filters:[
            {eq: {field: "tender.title", value: "Плитка керамічна"}},
            {eq: {field: "tender.description", value: "плитка керамічна"}},
            {eq: {field: "item.description", value: "Плитка керамічна Gress Атем Veneto, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
            {eq: {field: "item.unit.id", value: "MTK"}},
            {gte: {field: "probability", value: "0.0002"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }
''')
print(dumps(client.execute(query, variable_values=params), indent= 4))

{
    "Predictions": {
        "Classification": {
            "values": [
                {
                    "entity": {
                        "id": "44111700-8",
                        "description": "\u041a\u0430\u0445\u0435\u043b\u044c",
                        "scheme": "CPV"
                    },
                    "probability": 0.6110372
                },
                {
                    "entity": {
                        "id": "44111300-4",
                        "description": "\u041a\u0435\u0440\u0430\u043c\u0456\u0447\u043d\u0456 \u0432\u0438\u0440\u043e\u0431\u0438",
                        "scheme": "CPV"
                    },
                    "probability": 0.16699347
                },
                {
                    "entity": {
                        "id": "44110000-4",
                        "description": "\u041a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0456\u0439\u043d\u0456 \u043c\u0430\u0442\u0435\u0440\u0456\u0430\u043b\u0438",